In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from sklearn.metrics import f1_score
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import gensim.downloader
import pandas as pd
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score as seqeval_f1_score
from tqdm import tqdm
import time
import os
import multiprocessing
import numpy as np
import codecs
import re
import torch.nn.functional as F
from torchcrf import CRF
import time

In [2]:
num_cores = multiprocessing.cpu_count()
print(f"Number of CPU cores available: {num_cores}")

Number of CPU cores available: 16


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
pretrained_model = gensim.downloader.load('word2vec-google-news-300')

**Question 1.1**
Based on word2vec embeddings you have downloaded, use cosine similarity to find the most similar
word to each of these words: (a) “student”; (b) “Apple”; (c) “apple”. Report the most similar word
and its cosine similarity.

In [5]:
def most_similar_word(word): 
    try:
        similar_words = pretrained_model.most_similar(word, topn=1)
        for similar_word, score in similar_words:
            print(f"{similar_word}: {score}")
    except KeyError:
        print(f"'{word}' is not in the vocabulary.")

print("Most similar word to 'student':")
most_similar_word('student')

print("\nMost similar word to 'Apple':")
most_similar_word('Apple')

print("\nMost similar word to 'apple':")
most_similar_word('apple')

Most similar word to 'student':
students: 0.7294867038726807

Most similar word to 'Apple':
Apple_AAPL: 0.7456986308097839

Most similar word to 'apple':
apples: 0.720359742641449


**Question 1.2**

(a) Describe the size (number of sentences) of the training, development and test file for CoNLL2003.
Specify the complete set of all possible word labels based on the tagging scheme (IO, BIO,
etc.) you chose.

(b) Choose an example sentence from the training set of CoNLL2003 that has at least two named
entities with more than one word. Explain how to form complete named entities from the label
for each word, and list all the named entities in this sentence.

**Solution**

(a)

The size (number of sentences) of the training, development, and test files for CoNLL2003 is as follows:

| Training: 14,987 |
| Development: 3,466 |
| Test: 3,684 |

The complete set of all possible word labels based on the BIO tagging scheme is as follows:

1. O (Outside): Represents words that are not part of any named entity.

2. B-PER (Beginning of a Person entity): Marks the first word of a named entity that is a person.

3. I-PER (Inside of a Person entity): Marks words inside a named entity that is a person.

4. B-LOC (Beginning of a Location entity): Marks the first word of a named entity that is a location.

5. I-LOC (Inside of a Location entity): Marks words inside a named entity that is a location.

6. B-ORG (Beginning of an Organization entity): Marks the first word of a named entity that is an organization.

7. I-ORG (Inside of an Organization entity): Marks words inside a named entity that is an organization.

8. B-MISC (Beginning of a Miscellaneous entity): Marks the first word of a named entity that is miscellaneous or doesn't fit into the other categories.

9. I-MISC (Inside of a Miscellaneous entity): Marks words inside a named entity that is miscellaneous.

(b)

Here is an example sentence from the training set of CoNLL2003 that has at least two named entities with more than one word:

*The U.S. Federal Reserve is expected to raise interest rates next week.*

To form complete named entities from the label for each word, we need to group together all of the words that have the same label and are contiguous. In this example, the named entities are:

*U.S. Federal Reserve (ORG)*
*interest rates (MISC)*

Therefore, the complete set of named entities in this sentence is:

*U.S. Federal Reserve, interest rates*

In [6]:
train_file = "eng.train"
dev_file = "eng.testa"
test_file = "eng.testb"

In [7]:
def zero_digits(s):
    """
    Replace every digit in a string by a zero.
    """
    return re.sub('\d', '0', s)

def load_sentences(path, zeros):
    """
    Load sentences. A line must contain at least a word and its tag.
    Sentences are separated by empty lines.
    """
    sentences = []
    sentence = []
    for line in codecs.open(path, 'r', 'utf8'):
        line = zero_digits(line.rstrip()) if zeros else line.rstrip()
        if not line:
            if len(sentence) > 0:
                if 'DOCSTART' not in sentence[0][0]:
                    sentences.append(sentence)
                sentence = []
        else:
            word = line.split()
            assert len(word) >= 2
            sentence.append(word)
    if len(sentence) > 0:
        if 'DOCSTART' not in sentence[0][0]:
            sentences.append(sentence)
    return sentences

In [8]:
train_sentences = load_sentences(train_file, True)
test_sentences = load_sentences(test_file, True)
dev_sentences = load_sentences(dev_file, True)

In [9]:
def create_dico(item_list):
    """
    Create a dictionary of items from a list of list of items.
    """
    assert type(item_list) is list
    dico = {}
    dico['<PAD>'] = 1
    dico['<OOV>'] = 1
    for items in item_list:
        for item in items:
            if item not in dico:
                dico[item] = 1
            else:
                dico[item] += 1
    
    return dico
def create_mapping(dico):
    """
    Create a mapping (item to ID / ID to item) from a dictionary.
    Items are ordered by decreasing frequency.
    """
    sorted_items = sorted(dico.items(), key=lambda x: (-x[1], x[0]))
    id_to_item = {i: v[0] for i, v in enumerate(sorted_items)}
    item_to_id = {v: k for k, v in id_to_item.items()}
    return item_to_id, id_to_item
def char_mapping(sentences):
    """
    Create a dictionary and mapping of characters, sorted by frequency.
    """
    chars = ["".join([w[0] for w in s]) for s in sentences]
    dico = create_dico(chars)
    char_to_id, id_to_char = create_mapping(dico)
    return dico, char_to_id, id_to_char

dict_chars, char_to_id, id_to_char = char_mapping(train_sentences)
label_to_index = {"O": 0, "B-PER": 1, "I-PER": 2, "B-LOC": 3, "I-LOC": 4, "B-ORG": 5, "I-ORG": 6, "B-MISC": 7, "I-MISC": 8}
word_to_index = pretrained_model.key_to_index
OOV_INDEX = 3000000

In [10]:
class CoNLL2003Dataset(torch.utils.data.Dataset):
    def __init__(self, sentences_list, word_to_index, char_to_id, label_to_index, max_sequence_length, max_word_length):
        super(CoNLL2003Dataset, self).__init__()

        self.sentences = []  # List to store sentences
        self.labels = []     # List to store labels
        self.char_indices = []  # List to store character-level data

        for sent in sentences_list:
            words = []
            labels = []
            char_data = []  # Store character-level data
            for word in sent:
                words.append(word[0])
                labels.append(word[-1])
                # Convert word to character indices (replace with your method to map characters to indices)
                char_indices = [char_to_id.get(char,75) for char in word[0]]
                char_data.append(char_indices)
            # will append OOV_index if the word is not foind in word_to_index dictionary
            words_index = [int(word_to_index.get(word, OOV_INDEX)) for word in words]
            labels_index = [int(label_to_index[label]) for label in labels]

            # Ensure that char_data is of length max_word_length
            for i in range(len(char_data)):
                if len(char_data[i]) < max_word_length:
                    pad_length = max_word_length - len(char_data[i])
                    pad_left = pad_length // 2
                    pad_right = pad_length - pad_left
                    char_data[i] = [76] * pad_left + char_data[i] + [76] * pad_right
                elif len(char_data[i]) > max_word_length:
                    char_data[i] = char_data[i][:max_word_length]
            if len(words_index) < max_sequence_length:
                for i in range(len(words_index), max_sequence_length):
                    words_index.append(0)
                    labels_index.append(0)
                    char_data.append([76] * max_word_length)
            elif len(words_index) > max_sequence_length:
                words_index = words_index[:max_sequence_length]
                labels_index = labels_index[:max_sequence_length]
                char_data = char_data[:max_sequence_length]
            self.sentences.append(torch.LongTensor(words_index))
            self.labels.append(torch.LongTensor(labels_index))
            self.char_indices.append(torch.LongTensor(char_data))

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, index):
        return self.sentences[index], self.labels[index], self.char_indices[index]
train_dataset = CoNLL2003Dataset(train_sentences, word_to_index,char_to_id, label_to_index,100,15)
dev_dataset = CoNLL2003Dataset(dev_sentences, word_to_index,char_to_id, label_to_index,100,15)
test_dataset = CoNLL2003Dataset(test_sentences, word_to_index,char_to_id, label_to_index,100,15)

In [11]:
pretrained_embeddings =pretrained_model.vectors
pretrained_embeddings = np.append(pretrained_embeddings, [[0] *300], axis=0)
pretrained_embeddings = torch.FloatTensor(pretrained_embeddings)

In [12]:
class CharCNN(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, char_cnn_output_dim, kernel_sizes):
        super(CharCNN, self).__init__()

        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim)

        # Define convolutional layers with different kernel sizes
        self.conv_layers = nn.ModuleList([nn.Conv2d(1, char_cnn_output_dim, (k, char_embedding_dim)) for k in kernel_sizes])

    def forward(self, char_sequences):
        batch_size, sequence_length, word_length = char_sequences.size()

        # Embed characters
        char_embeddings = self.char_embedding(char_sequences)

        # Reshape the embeddings to have 1 channel
        char_embeddings = char_embeddings.view(batch_size * sequence_length, 1, word_length, -1)

        # Apply convolutional layers
        conv_outputs = [conv(char_embeddings).squeeze(3) for conv in self.conv_layers]

        # Max-pooling over time
        pooled_outputs = [nn.functional.max_pool1d(conv, conv.size(2)).squeeze(2) for conv in conv_outputs]

        # Concatenate the outputs from different kernel sizes
        char_cnn_output = torch.cat(pooled_outputs, 1)

        # Reshape the output to be 2D (batch_size, char_cnn_output_dim)
        char_cnn_output = char_cnn_output.view(batch_size, sequence_length, -1)

        return char_cnn_output

In [13]:
class NERModel(nn.Module):
    def __init__(self, embedding_dim, final_vector_dim, pretrained_embeddings, char_cnn,num_labels):
        super(NERModel, self).__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=True)
        self.char_cnn = char_cnn  # Character-level embedding module
        self.bilstm = nn.LSTM((embedding_dim + char_cnn_output_dim), hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(2 * hidden_dim, num_labels)
        self.crf = CRF(num_labels) 
        
    def forward(self, word_inputs, char_inputs):
        # Word embeddings
        word_embeddings = self.embedding_layer(word_inputs)
        char_embeddings = self.char_cnn(char_inputs)
        combined_embeddings = torch.cat([word_embeddings, char_embeddings], dim=2)
   

        # BiLSTM layer
        lstm_out, _ = self.bilstm(combined_embeddings)
        final_vector_representations = self.fc(lstm_out)
        # emissions = self.crf(final_vector_representations)

        return final_vector_representations

In [18]:
char_vocab_size = len(dict_chars.keys())  # Adjust based on your character vocabulary size
char_embedding_dim = 100

# Hyperparameters
num_epochs = 10
learning_rate = 0.001
batch_size = 16

hidden_dim = 256  # Hidden dimension for the BiLSTM layer
num_layers = 1  # Number of BiLSTM layers
num_filters = 64 # For CNN Dimension
kernel_sizes = [3] # Number of CNN layers and the kernal_size
char_cnn_output_dim = num_filters * len(kernel_sizes)

# Mini-batch data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=batch_size )
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

char_cnn = CharCNN(char_vocab_size, char_embedding_dim, num_filters, kernel_sizes)
model = NERModel(embedding_dim=300,final_vector_dim=300,pretrained_embeddings=pretrained_embeddings,char_cnn=char_cnn,num_labels=len(label_to_index.keys())).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

best_dev_loss = 100000
best_model_state = model.state_dict()

In [19]:
def evaluate_model(model, dataloader, device):
    model.eval()
    predictions = []
    labels = []
    losses = []

    with torch.no_grad():
        for inputs, true_labels, char_embeddings in dataloader:
            inputs, true_labels, char_embeddings = inputs.to(device), true_labels.to(device), char_embeddings.to(device)
            emissions = model(inputs, char_embeddings)
            loss = -model.crf(emissions, true_labels)
            losses.append(loss.item())
            
            tags = np.array(model.crf.decode(emissions)).T
            predictions.extend(tags)
            labels.extend(true_labels.tolist())

        converted_predictions = []
        converted_labels = []

        for sent in predictions:
            converted_sent = [list(label_to_index.keys())[list(label_to_index.values()).index(int(i))] for i in sent]
            converted_predictions.append(converted_sent)

        for sent in labels:
            converted_sent = [list(label_to_index.keys())[list(label_to_index.values()).index(int(i))] for i in sent]
            converted_labels.append(converted_sent)

        f1 = seqeval_f1_score(converted_labels, converted_predictions)
        avg_loss = sum(losses) / len(losses)

    return avg_loss, f1

In [16]:
time_begin = time.time()
for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}', unit='batch', leave=False)
    losses = []

    for inputs, labels, char_embeddings in progress_bar:
        inputs, labels, char_embeddings = inputs.to(device), labels.to(device), char_embeddings.to(device)
        optimizer.zero_grad()
        emissions = model(inputs, char_embeddings)
        loss = -model.crf(emissions, labels)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss=loss.item())

    train_loss = sum(losses) / len(losses)

    dev_loss, dev_f1 = evaluate_model(model, dev_loader, device)

    print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Val Loss: {dev_loss:.4f}, Val F1: {dev_f1:.4f}')

    if dev_loss < best_dev_loss:
        best_dev_loss = dev_loss
        torch.save(model, 'best_model_cnn_crf_300charemb.pth')
time_end = time.time()
time_taken = time_end - time_begin
print("Time Taken: " , time_taken, "s")

Epoch 1:   0%|                                                                              | 0/878 [00:00<?, ?batch/s]C:\Users\kaksh\anaconda3\envs\torchenv\lib\site-packages\torchcrf\__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorCompare.cpp:493.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch 1, Train Loss: 54.1384, Val Loss: 18.6494, Val F1: 0.8595


Epoch 2, Train Loss: 14.5006, Val Loss: 14.9281, Val F1: 0.8776


Epoch 3, Train Loss: 9.7628, Val Loss: 13.0431, Val F1: 0.9033


Epoch 4, Train Loss: 7.0452, Val Loss: 13.0799, Val F1: 0.8990


...

Epoch 6, Train Loss: 3.3503, Val Loss: 12.5246, Val F1: 0.9128


Epoch 7, Train Loss: 2.2276, Val Loss: 12.9516, Val F1: 0.9124


Epoch 8, Train Loss: 1.5175, Val Loss: 15.1481, Val F1: 0.9071


Epoch 9, Train Loss: 1.4280, Val Loss: 15.6012, Val F1: 0.9049


Epoch 10, Train Loss: 1.1308, Val Loss: 17.0144, Val F1: 0.9080
Time Taken:  397.27873635292053 s


In [20]:
model = torch.load('best_model_cnn_crf.pth')
dev_loss,dev_f1 = evaluate_model(model, dev_loader, device)
test_loss,test_f1 = evaluate_model(model, test_loader, device)
print(f'Test F1: {test_f1:.4f}, Test Loss: {test_loss:.4f}, Val Loss: {dev_loss:.4f}, Val F1: {dev_f1:.4f}')

C:\Users\kaksh\anaconda3\envs\torchenv\lib\site-packages\torchcrf\__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorCompare.cpp:493.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Test F1: 0.8635, Test Loss: 22.0705, Val Loss: 12.5246, Val F1: 0.9128
